* analizar imagenes 

In [1]:
import os
import string
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.filters import convolve
espacioTrabajo = os.getcwd()
import time
import timeit
from collections import deque
from memory_profiler import memory_usage
directorioActual =  espacioTrabajo.replace(string.punctuation[23], string.punctuation[14]) #Directorio actual reemplazando "\" por "/"

In [2]:
class Imagen:
    nombre = None  # Nombre archivo de la imagen
    ubicacion = None  # Direccion en la que se halla la imagen
    matriz_bytes = None  # Matriz de numpy que contiene bytes de cada imagen

    def __init__(self, nombre, ubicacion):
        self.nombre = nombre
        self.ubicacion = ubicacion
        self.matriz_bytes = None

    def generar_ubicacion(self, nueva_ubicacion: str):
        self.ubicacion = nueva_ubicacion

    def calculo_energia(self):
        filtro_horizontal = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
        filtro_vertical = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
        # Se aplica la convulución respecto a los filtros.
        convol_horizontal = convolve(self.matriz_bytes, filtro_horizontal)
        convol_vertical = convolve(self.matriz_bytes, filtro_vertical)
        # Esta operacion nos devuelve un mapa de energia para matrices con valores en escala de grises.
        return np.absolute(convol_horizontal) + np.absolute(convol_vertical)

    def cargarImagen(self):
        print("cargando... "+self.nombre+" como formato cvs")
        reader = open(self.ubicacion+self.nombre, 'r')
        self.matriz_bytes = np.loadtxt(reader, delimiter=",")

    def mostrarImagen(self):
        plt.figure(figsize=(10, 10))
        plt.grid("off")
        plt.axis("off")
        # Recibe la matriz de cada imagen en bytes con valores de 0 a 255.
        plt.imshow(self.matriz_bytes, cmap="gray", vmin=0, vmax=255)
        plt.show()

    def minimaCostura(self):
        r, c = self.matriz_bytes.shape
        mapaEnergia = self.calculo_energia()
        M = mapaEnergia.copy()
        retroceso = np.zeros_like(M, dtype=np.int)
        for i in range(1, r):
            for j in range(0, c):
                if j == 0:
                    idx = np.argmin(M[i - 1, j:j + 2])
                    retroceso[i, j] = idx + j
                    min_energy = M[i - 1, idx + j]
                else:
                    idx = np.argmin(M[i - 1, j - 1:j + 2])
                    retroceso[i, j] = idx + j - 1
                    min_energy = M[i - 1, idx + j - 1]
                M[i, j] += min_energy
        return M, retroceso

    def cortarColumna(self):
        r, c = self.matriz_bytes.shape
        M, retroceso = self.minimaCostura()
        filtro = np.ones((r, c), dtype=np.bool)
        j = np.argmin(M[-1])
        for i in reversed(range(r)):
            filtro[i, j] = False
            j = retroceso[i, j]
        self.matriz_bytes = self.matriz_bytes[filtro]
        self.matriz_bytes = self.matriz_bytes.reshape(r, c-1)

    def cortarFila(self, scale):
        self.matriz_bytes = np.rot90(self.matriz_bytes, 1)
        self.eliminandoColumnas(scale)
        self.matriz_bytes = np.rot90(self.matriz_bytes, 3)
    
    def eliminandoColumnas(self, scale_c):
        
        r, c = self.matriz_bytes.shape
        new_c = int(scale_c * c)
        for i in range(c - new_c):
            
            self.cortarColumna()

    def resumirImagen(self):
        r, c = self.matriz_bytes.shape
        for i in range(r):
            for j in range(c):
                self.matriz_bytes[i][j] = (self.matriz_bytes[i][j]//50)*50

    def recorridoLongitudCompresion(self):
        Datos = [] #datos comprimidos no se puede dque por el metodo que se esta usamdo para guardar la matriz
        for i in range(len(self.matriz_bytes)): #cyclo para recorrer filas de la matriz
            Estamos = self.matriz_bytes[i]#señalar la matriz original
            n = len(Estamos)
            j= 0
            SalidaDeLaFila = []#arreglo para guar la salida de cada fila
            while j<= n - 1: #ciclo para recorrer todos los caracteres
                contador = 1
                while (j< n - 1 and Estamos[j] == Estamos[j+ 1]):#ciclo para contar los caracteres repetidos
                    contador += 1
                    j+= 1
                j+= 1#se suma despes de terminar el ciclo para iniciar el proximo desde un caracter nuevo
                
                if contador > 1:
                    SalidaDeLaFila.append("$"+str(contador)) #añadir repeticiones silas hay
                SalidaDeLaFila.append(Estamos[j- 1])  #añadir el caracter
            
            Datos.append(SalidaDeLaFila)#guerdar en la matriz

        return Datos

def guardarImagenComprimida(matriz,imagen,carpeta):
    np.savetxt("codigo/datasets/csv/"+carpeta+"/ImagenComprimida" + str(imagen.nombre), matriz,fmt='%s', delimiter=",")
def descomprimirRecorridoDeLongitud(imagen):
    
    nuevaImagen=[]
    for i in range(len(imagen)):
        estamos=imagen[i]
        n=len(estamos)
        salida_de_linea=[]
        j=0
        while j<n:
            repetir=str(estamos[j])
            if repetir[0]=='$':
                rellenar=int(float(estamos[j+1]))
                repetir=int(repetir[1:])
                for k in range(repetir):
                    salida_de_linea.append(rellenar) 
                j+=1
            else:
                salida_de_linea.append(int(float(repetir)))
            j+=1
        nuevaImagen.append(salida_de_linea)
    return nuevaImagen


In [ ]:
def ejecutarSeamCarving(img):
    img.eliminandoColumnas(0.98)

In [ ]:
def ejecutarRunLength(img,decodificado):
    img.resumirImagen()      
    decodificado[0]=img.recorridoLongitudCompresion()

In [ ]:

lista_imagenes_Enfermos = os.listdir(directorioActual+'/codigo/datasets/csv/enfermo_csv/')#cambiar enfermo_csv por enfermo_csv
#Cargando imagenes de animales enfermos
c=0
listaTiempos2=deque()

for imagen in lista_imagenes_Enfermos:
    decodificado=[1]
    img = None
    memoriaPrograma=memory_usage(-1)[0]
    img=Imagen(imagen,directorioActual+'/codigo/datasets/csv/enfermo_csv/')#cambiar zzz por enfermo_csv
    img.cargarImagen()
    
    #Ejecutando Seam Carving
    start1 = time.time()               
    memoriaSeamCarving = memory_usage((ejecutarSeamCarving, (img,),), timestamps=False, max_usage=True)
    end1 = time.time() 
    #Termina seam carving
    
    memoriaSeamCarvingNeta=memoriaSeamCarving-memoriaPrograma
    ejecucionSeam=end1-start1
    
    memoriaPrograma=None
    memoriaPrograma=memory_usage(-1)[0]
    #Ejecutando RunLength
    start2= time.time()
    memoriaRunLength = memory_usage((ejecutarRunLength, (img,decodificado),), timestamps=False, max_usage=True)
    end2=time.time()
    #Termina RunLength
    
    
    memoriaRunLenthNeta = memoriaRunLength-memoriaPrograma
    ejecucionRunLength=end2 - start2

    memoriaCompresion = memoriaRunLenthNeta + memoriaSeamCarvingNeta
    print("simp")
    print(memoriaSeamCarvingNeta)
    print("len")
    print(memoriaRunLenthNeta)
    print("total")
    print(memoriaCompresion)

    guardarImagenComprimida(decodificado[0],img,"enfermoComprimido")
    
    #nombreimagen,tiempo ejecucion de compresion,toriginal, tcomprimido
    listaTiempos2.append((imagen+":"+str(ejecucionSeam)+":"+str(ejecucionRunLength)+":"+str(ejecucionSeam+ejecucionRunLength)+":"+str(memoriaSeamCarvingNeta)+":"+str(memoriaRunLenthNeta)+":"+str(memoriaCompresion)))
    #descomp=descomprimirRecorridoDeLongitud(decodificado)
    #print(descomprimirRecorridoDeLongitud(decodificado))
    #plt.figure(figsize=(10, 10))
    #plt.grid("off")
    #plt.axis("off")
    # Recibe la matriz de cada imagen en bytes con valores de 0 a 255.
    #plt.imshow(descomp, cmap="gray", vmin=0, vmax=255)
    #plt.show()
    c+=1
    if c==2:
        break
archivoTiempo= open("datosRunEnfermos.txt","w")
archivoTiempo.write("nombreImagen:tEjecucionSeam:tEjecucionRun:tEtotal:memoriaSeamCarvingNeta:memoriaRunLenthNeta:memoriaCompresion"+"\n")
for linea in listaTiempos2:
    archivoTiempo.write(linea+"\n")

archivoTiempo.close()  

In [ ]:

lista_imagenes_Sanos= os.listdir(directorioActual+'/codigo/datasets/csv/sano_csv/')
#Cargando imagenes de animales sanos
c=0
listaTiempos3=deque()

for imagen in lista_imagenes_Sanos:
    decodificado=[1]
    img = None
    memoriaPrograma=memory_usage(-1)[0]
    img=Imagen(imagen,directorioActual+'/codigo/datasets/csv/sano_csv/')#cambiar zzz por enfermo_csv
    img.cargarImagen()
    
    #Ejecutando Seam Carving
    start1 = time.time()               
    memoriaSeamCarving = memory_usage((ejecutarSeamCarving, (img,),), timestamps=False, max_usage=True)
    end1 = time.time() 
    #Termina seam carving
    
    memoriaSeamCarvingNeta=memoriaSeamCarving-memoriaPrograma
    ejecucionSeam=end1-start1
    
    memoriaPrograma=None
    memoriaPrograma=memory_usage(-1)[0]
    #Ejecutando RunLength
    start2= time.time()
    memoriaRunLength = memory_usage((ejecutarRunLength, (img,decodificado),), timestamps=False, max_usage=True)
    end2=time.time()
    #Termina RunLength
    
    
    memoriaRunLenthNeta = memoriaRunLength-memoriaPrograma
    ejecucionRunLength=end2 - start2

    memoriaCompresion = memoriaRunLenthNeta + memoriaSeamCarvingNeta
    print("simp")
    print(memoriaSeamCarvingNeta)
    print("len")
    print(memoriaRunLenthNeta)
    print("total")
    print(memoriaCompresion)

    guardarImagenComprimida(decodificado[0],img,"sanoComprimido")
    #nombreimagen,tiempo ejecucion de compresion,toriginal, tcomprimido
    listaTiempos3.append((imagen+":"+str(ejecucionSeam)+":"+str(ejecucionRunLength)+":"+str(ejecucionSeam+ejecucionRunLength)+":"+str(memoriaSeamCarvingNeta)+":"+str(memoriaRunLenthNeta)+":"+str(memoriaCompresion)))
    #descomp=descomprimirRecorridoDeLongitud(decodificado)
    #print(descomprimirRecorridoDeLongitud(decodificado))
    #plt.figure(figsize=(10, 10))
    #plt.grid("off")
    #plt.axis("off")
    # Recibe la matriz de cada imagen en bytes con valores de 0 a 255.
    #plt.imshow(descomp, cmap="gray", vmin=0, vmax=255)
    #plt.show()
    c+=1
    if c==2:
        break
    
archivoTiempo= open("datosRunSanos.txt","w")
archivoTiempo.write("nombreImagen:tEjecucionSeam:tEjecucionRun:tEtotal:memoriaSeamCarvingNeta:memoriaRunLenthNeta:memoriaCompresion"+"\n")
for linea in listaTiempos3:
    archivoTiempo.write(linea+"\n")

archivoTiempo.close()  

In [3]:
def cargarComprimido(imagen,carpeta):
    reader = open(directorioActual+'/codigo/datasets/csv/'+carpeta+'/'+imagen, 'r')
    linea= reader.readline()
    lista=[]
    while linea!='':
        lista.append(eval(linea))
        linea=reader.readline()
    reader.close()
    return lista

In [4]:
def descomprimir(imagen,arreglo):
    arreglo[0]=descomprimirRecorridoDeLongitud(imagen)

In [5]:
lista_imagenes_Enfermos_comprimidos = os.listdir(directorioActual+'/codigo/datasets/csv/enfermoComprimido/')#cambiar enfermo_csv por enfermo_csv
listaDecompresion=deque()
c=0
for imagenes in lista_imagenes_Enfermos_comprimidos:
    imagenComprimidaMatriz=None
    imagenDescomprimida=None
    memoriaPrograma=memory_usage(-1)[0]
    imagenComprimidaMatriz=cargarComprimido(imagenes,"enfermoComprimido")
    tiempoDescompresionInicial=time.time()
    imagenDescomprimida=[1]
    memoriaDescompresion=memory_usage((descomprimir, (imagenComprimidaMatriz,imagenDescomprimida),), timestamps=False, max_usage=True)
    tiempoDEscompresionFinal=time.time()
    listaDecompresion.append(imagenes+":"+str(tiempoDEscompresionFinal-tiempoDescompresionInicial)+":"+str(memoriaDescompresion-memoriaPrograma))
    c+=1
    if c==2:
        break
archivoDescompresion= open("datosDescompresionEnfermos.txt","w")
archivoDescompresion.write("nombreImagen:tiempo:memoria"+"\n")
for linea in listaDecompresion:
    archivoDescompresion.write(linea+"\n")

archivoDescompresion.close()  

In [ ]:
lista_imagenes_Sanos_comprimidos = os.listdir(directorioActual+'/codigo/datasets/csv/sanoComprimido/')#cambiar sano_csv por enfermo_csv
listaDecompresion2=deque()
c=0
for imagenes in lista_imagenes_Sanos_comprimidos:
    imagenComprimidaMatriz=None
    imagenDescomprimida=None
    memoriaPrograma=memory_usage(-1)[0]
    imagenComprimidaMatriz=cargarComprimido(imagenes,"sanoComprimido")
    tiempoDescompresionInicial=time.time()
    imagenDescomprimida=[1]
    memoriaDescompresion=memory_usage((descomprimir, (imagenComprimidaMatriz,imagenDescomprimida),), timestamps=False, max_usage=True)
    tiempoDEscompresionFinal=time.time()
    listaDecompresion2.append(imagenes+":"+str(tiempoDEscompresionFinal-tiempoDescompresionInicial)+":"+str(memoriaDescompresion-memoriaPrograma))
    c+=1
    if c==2:
        break
archivoDescompresion= open("datosDescompresionSanos.txt","w")
archivoDescompresion.write("nombreImagen:tiempo:memoria"+"\n")
for linea in listaDecompresion2:
    archivoDescompresion.write(linea+"\n")

archivoDescompresion.close()  